# 🧠 Product Category Classification – Product Title Based ML Model

Ovaj notebook prikazuje kompletan tok rada za razvoj modela masinskog ucenja
koji automatski predlaze kategoriju proizvoda na osnovu njegovog naslova
(*Product Title*).

Cilj:
- Ucitati i analizirati skup podataka `products.csv`
- Pripremiti i ocistiti podatke za modeliranje
- Napraviti smislen **feature engineering** na osnovu naslova proizvoda
- Uporediti vise ML modela za klasifikaciju kategorija
- Izabrati najbolji model, evaluirati ga i sacuvati kao `.pkl` pipeline

Ovaj notebook je oslonac za skriptove:
- `src/train_model.py` – treniranje i cuvanje modela
- `src/predict_category.py` – interaktivno testiranje novog naslova proizvoda


## ⚙️ Uvoz biblioteka i osnovne postavke

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

import joblib

# Prikaz svih kolona u DataFrame-u
pd.set_option("display.max_columns", None)

# Stil za grafike
sns.set(style="whitegrid")


## 📚 Ucitavanje podataka

Ucitavamo skup podataka `products.csv` koji se nalazi u folderu `../data/`
u odnosu na ovu radnu svesku (koja je smestena u `notebook/` direktorijumu).


In [ ]:
# Putanja do CSV fajla relativno u odnosu na notebook
data_path = os.path.join("..", "data", "products.csv")

df_raw = pd.read_csv(data_path)

print("Dimenzije originalnog skupa:", df_raw.shape)
df_raw.head()


## 🔎 Struktura kolona, tipovi podataka, broj nedostajucih vrednosti

In [ ]:
df_raw.info()
print("\nBroj nedostajucih vrednosti po kolonama:")
print(df_raw.isna().sum())


## 🧹 Priprema i ciscenje podataka

Prvi korak je da standardizujemo nazive kolona (uklanjanje razmaka na pocetku/kraju,
pretvaranje u `snake_case`) i da sredimo osnovna polja:

- fokus je na `Product Title` kao glavnom tekstualnom feature-u,
- `Category Label` je ciljna promenljiva,
- dodatne kolone (`Number_of_Views`, `Merchant Rating`, datum) mozemo koristiti
  kao numeričke feature-e nakon obrade.

U nastavku vrsimo:
- preimenovanje kolona u dosledan format,
- uklanjanje redova sa nedostajucim naslovom ili kategorijom.


In [ ]:
# Kopija originalnog DataFrame-a za dalju obradu
df = df_raw.copy()

# Standardizacija naziva kolona
df = df.rename(columns=lambda c: c.strip())  # ukloni leading/trailing razmake
df = df.rename(columns=lambda c: c.lower().replace(" ", "_"))

# Specijalno sredimo naziv kolone za product_code (ako je imala underscore)
if "_product_code" in df.columns:
    df = df.rename(columns={"_product_code": "product_code"})

df.head()





In [ ]:
# provera kolona nakon preimenovanja
print("Kolone nakon preimenovanja:")
print(df.columns.tolist())

print("\nBroj nedostajucih vrednosti:")
print(df.isna().sum())


## 🧹 Sve sto nema naslov ili kategoriju – izbacujemo

In [ ]:
# Drop redova bez naslova ili kategorije
before_rows = df.shape[0]

df = df.dropna(subset=["product_title", "category_label"])

after_rows = df.shape[0]
print(f"Uklonjeno redova bez naslova ili kategorije: {before_rows - after_rows}")
print("Nove dimenzije skupa:", df.shape)


## 🪄 Numericka polja popunjujemo median vrednostima, datume pretvaramo u datetime.

In [ ]:
# Popunjavanje numerickih kolona median vrednostima
if "number_of_views" in df.columns:
    df["number_of_views"] = df["number_of_views"].fillna(df["number_of_views"].median())

if "merchant_rating" in df.columns:
    df["merchant_rating"] = df["merchant_rating"].fillna(df["merchant_rating"].median())

# Pretvaranje listing_date u datetime
if "listing_date" in df.columns:
    df["listing_date"] = pd.to_datetime(df["listing_date"], errors="coerce")
    # Nedostajuce datume popunimo najcescim (mode)
    if df["listing_date"].isna().sum() > 0:
        most_common_date = df["listing_date"].mode()[0]
        df["listing_date"] = df["listing_date"].fillna(most_common_date)

df.info()


## 🔎 Analiza ciljne promenljive (Category Label)

Pre nego sto predjemo na model, vazno je da razumemo kako su kategorije rasporedjene
(ucesce po klasama). Ovo nam pomaze da vidimo da li postoji jaka neuravnotezenost,
sto moze uticati na izbor modela i metrike.

In [ ]:
category_counts = df["category_label"].value_counts().sort_values(ascending=False)

print("Broj proizvoda po kategoriji (Top 20):")
print(category_counts.head(20))

plt.figure(figsize=(10, 6))
category_counts.head(10).plot(kind="bar")
plt.title("Top 10 kategorija po broju proizvoda")
plt.xlabel("Kategorija")
plt.ylabel("Broj proizvoda")
plt.tight_layout()
plt.show()


## 📁 Feature engineering

Osnovni ulaz u model je tekstualni naslov proizvoda (`product_title`).

Da bismo modelu dali više informacija, uvodimo dodatne feature-e:

- `title_char_len` – broj karaktera u naslovu,
- `title_word_count` – broj reci u naslovu,
- `title_digit_count` – broj cifara u naslovu,
- `has_digits` – indikator da li naslov sadrzi bar jednu cifru,
- `has_upper_acronym` – indikator prisustva akronima (npr. USB, LED),
- `number_of_views` – (ako postoji) broj pregleda,
- `merchant_rating` – (ako postoji) ocena prodavca.

Ovi feature-i ce biti kombinovani sa TF-IDF reprezentacijom teksta naslova.


In [ ]:
# Duzina naslova u karakterima
df["title_char_len"] = df["product_title"].str.len()

# Broj reci u naslovu
df["title_word_count"] = df["product_title"].str.split().str.len()

# Broj cifara u naslovu
df["title_digit_count"] = df["product_title"].str.count(r"\d")

# Da li naslov sadrzi cifru (0/1)
df["has_digits"] = (df["title_digit_count"] > 0).astype(int)

# Da li sadrzi akronim velikim slovima (npr. USB, LED, UHD...)
df["has_upper_acronym"] = df["product_title"].str.contains(r"\b[A-Z]{2,}\b", regex=True).astype(int)

df[["product_title", "title_char_len", "title_word_count", "title_digit_count", "has_digits", "has_upper_acronym"]].head()


## 🔪 Podela na trening i test skup

Kao ulazne promenljive (**X**) koristicemo:
- tekstualni naslov (`product_title`),
- dodatne numericke feature-e koje smo definisali,
- opcione numericke kolone poput `number_of_views` i `merchant_rating`.

Ciljna promenljiva (**y**) je `category_label`.

Skup delimo na trening (80%) i test (20%), sa `stratify` po kategoriji
kako bismo zadrzali slicnu raspodelu klasa u oba skupa.


In [ ]:
# Definisemo liste feature-a
text_feature = "product_title"

numeric_features = [
    "title_char_len",
    "title_word_count",
    "title_digit_count",
    "has_digits",
    "has_upper_acronym",
]

# Ako postoje ove kolone, dodaj ih u numericke feature-e
for col in ["number_of_views", "merchant_rating"]:
    if col in df.columns:
        numeric_features.append(col)

X = df[[text_feature] + numeric_features].copy()
y = df["category_label"].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


## ⚙️ Preprocessing pipeline

Za pripremu podataka koristicemo `ColumnTransformer`:

- Na tekstualni deo (`product_title`) primenjujemo **TF-IDF vektorizaciju**,
- Numericke feature-e standardizujemo sa `StandardScaler`.

Ovaj preprocesor ce biti zajednicki za sve modele, kako bismo mogli da ih
fer uporedimo i kasnije lako spakujemo ceo pipeline u `.pkl` fajl.


In [ ]:
text_transformer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=5,
    max_features=50000
)

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, text_feature),
        ("num", numeric_transformer, numeric_features),
    ]
)


## 🧠 Treniranje i poredjenje vise modela

Testiracemo tri modela:

- **Logistic Regression (multinomial)** – cesto vrlo jak baseline za tekst,
- **LinearSVC** – robusan model za visoko-dimenzionalni tekst,
- **RandomForestClassifier** – nen-linearni model za poredjenje.

Za svaki model:
- treniramo pipeline (preprocessor + classifier),
- merimo `accuracy` na test skupu,
- prikazujemo `classification_report`.


In [ ]:
models = {
    "LogisticRegression": LogisticRegression(
        max_iter=200,
        n_jobs=-1,
        multi_class="multinomial"
    ),
    "LinearSVC": LinearSVC(),
    "RandomForest": RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),
}

results = []
trained_pipelines = {}

for name, clf in models.items():
    print("=" * 80)
    print(f"Treniranje modela: {name}")
    print("=" * 80)
    
    pipe = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("classifier", clf),
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy na test skupu: {acc:.4f}\n")
    print("Classification report:\n")
    print(classification_report(y_test, y_pred))
    
    results.append({"model": name, "accuracy": acc})
    trained_pipelines[name] = pipe

results_df = pd.DataFrame(results).sort_values(by="accuracy", ascending=False)
results_df


## 📊 Izbor finalnog modela

Na osnovu dobijenih rezultata biramo model sa najboljom tacnoscu na test skupu.
U nastavku ćemo:

- izdvojiti najbolji model,
- detaljnije analizirati njegovu konfuzionu matricu,
- zatim ga istrenirati na celom skupu podataka i sacuvati kao `.pkl` pipeline.


In [ ]:
best_row = results_df.iloc[0]
best_model_name = best_row["model"]
best_accuracy = best_row["accuracy"]

print(f"Najbolji model: {best_model_name} (accuracy = {best_accuracy:.4f})")

best_pipeline = trained_pipelines[best_model_name]


In [ ]:
# Predikcije najboljeg modela
y_pred_best = best_pipeline.predict(X_test)

labels = np.unique(y_test)

cm = confusion_matrix(y_test, y_pred_best, labels=labels)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.title(f"Confusion Matrix – {best_model_name}")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.tight_layout()
plt.show()


## ⚙️ Treniranje finalnog pipeline-a na celom skupu i cuvanje modela

Za produkcionu upotrebu, finalni model je uobicajeno treniran na celom
dostupnom skupu podataka (ne samo na trening delu), kako bi imao sto vise informacija.

Koraci:
- napravimo novi pipeline sa istim preprocesorom i izabranim algoritmom,
- istreniramo ga na celom skupu (`X`, `y`),
- sacuvamo ga u fajl: `../models/product_category_model.pkl`.


In [ ]:
# Ponovo uzimamo "goli" classifier za najbolji model
final_clf = models[best_model_name]

final_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", final_clf),
])

final_pipeline.fit(X, y)

models_dir = os.path.join("..", "models")
os.makedirs(models_dir, exist_ok=True)

model_path = os.path.join(models_dir, "product_category_model.pkl")

joblib.dump(final_pipeline, model_path)

print(f"Finalni pipeline sačuvan u fajl: {model_path}")


## 💾 Cuvanje rezultata poredjenja modela

Radi transparentnosti i kasnije analize, rezultate poredjenja modela
cuvamo u fajl `../results/model_comparison.txt`.

Ovaj fajl ce koristiti i skript `train_model.py` kao referencu
za evaluaciju modela pri treniranju.


In [ ]:
results_dir = os.path.join("..", "results")
os.makedirs(results_dir, exist_ok=True)

comparison_path = os.path.join(results_dir, "model_comparison.txt")

with open(comparison_path, "w") as f:
    f.write("Model comparison (accuracy on test set)\n")
    f.write("-" * 50 + "\n")
    for _, row in results_df.iterrows():
        f.write(f"{row['model']}: {row['accuracy']:.4f}\n")

print(f"Rezultati poredjenja modela sacuvani u: {comparison_path}")


## ⚒️ Rucno testiranje primera iz zadatka

Na kraju, rucno testiramo nekoliko primera iz zadatka
kako bismo proverili ponasanje modela:

- *iphone 7 32gb gold,4,3,Apple iPhone 7 32GB* → ocekujemo `Mobile Phones`
- *olympus e m10 mark iii geh use silber* → ocekujemo `Digital Cameras`
- *kenwood k20mss15 solo* → ocekujemo `Microwaves`
- *bosch wap28390gb 8kg 1400 spin* → ocekujemo `Washing Machines`
- *bosch serie 4 kgv39vl31g* → ocekujemo `Fridge Freezers`
- *smeg sbs8004po* → ocekujemo `Fridge Freezers`


In [ ]:
test_titles = [
    "iphone 7 32gb gold,4,3,Apple iPhone 7 32GB",
    "olympus e m10 mark iii geh use silber",
    "kenwood k20mss15 solo",
    "bosch wap28390gb 8kg 1400 spin",
    "bosch serie 4 kgv39vl31g",
    "smeg sbs8004po",
]

test_df = pd.DataFrame({ "product_title": test_titles })

# Moramo generisati i numericke feature-e kao sto smo radili i za trening skup
test_df["title_char_len"] = test_df["product_title"].str.len()
test_df["title_word_count"] = test_df["product_title"].str.split().str.len()
test_df["title_digit_count"] = test_df["product_title"].str.count(r"\d")
test_df["has_digits"] = (test_df["title_digit_count"] > 0).astype(int)
test_df["has_upper_acronym"] = test_df["product_title"].str.contains(r"\b[A-Z]{2,}\b", regex=True).astype(int)

# Numericke kolone koje nisu prisutne ovde (views/rating) popunjavamo nulama
for col in ["number_of_views", "merchant_rating"]:
    if col in numeric_features and col not in test_df.columns:
        test_df[col] = 0.0

preds = final_pipeline.predict(test_df[[text_feature] + numeric_features])

for title, cat in zip(test_titles, preds):
    print(f"'{title}' -> predvidjena kategorija: {cat}")
